### 🏛️ Parliament Chart – Visualizing Seats in a Legislative Assembly

A **Parliament Chart** displays the distribution of seats among parties or groups in a semi-circular layout, resembling a legislative chamber.  
It’s commonly used to visualize **election results**, **coalition dynamics**, or **party representation** in parliaments or councils.


In [10]:
import plotly.express as px
import pandas as pd
import math
import plotly.express as px

def Sort_Tuple(tup):
    return sorted(tup, key=lambda x: x[1])

def parlamentary_Coord(df,angle_total=180,rows=4, ratio=6,initial='NAME'):    
    arco_total = 0
    angles = []
    for i in range(rows):
        arco_total += math.pi * int(ratio+i)
    for i in range(rows):
        arco_radio = math.pi * int(ratio+i)
        angles.append(angle_total/round(arco_radio/(arco_total/len(df)),0))
    coord = []
    for a in range(len(angles)):
        current_angle = angles[a]/2
        for i in range(int(round(angle_total/angles[a],0))):        
            coord.append((ratio+a, current_angle))
            current_angle += angles[a]
    coord = Sort_Tuple(coord)    
    df["radio"] = list(zip(*coord))[0]
    df["tetha"] = list(zip(*coord))[1]
    df["INITIAL"] = df[initial].apply(lambda x: x[0]) # only for text in marker chart
    return df

url = 'parlamento.csv'
df = pd.read_csv(url, sep=';')
df = df.iloc[:] #slice for some test. some errors will be generated depending on the length of the df and the configuration of the chart
df = df.sort_values(by=['PARTIDO'], ascending=True) # remember sort df for column 

#config for parlamentary plot
graph_style = "plotly_dark" #template plotly
height = 500 # height plot
angle_total = 240 # all cover plot, recomended 90 - 300
size_marker = 20  #config by size angle_total in design

#df your dataframe
df = parlamentary_Coord(df,angle_total,6,6)

angle_start = (180 - angle_total)/2
angle_end = 180 + (angle_total - 180)/2

# recomended use 'PARTIDO', 'COLOR' in your dataframe 
d_ict = df[["PARTIDO", "COLOR"]]
d_ict = d_ict.drop_duplicates()
dct = dict(d_ict.values)
fig = px.scatter_polar(
    df,
    r="radio",
    theta="tetha",
    color="PARTIDO",
    color_discrete_map=dct,
    text="INITIAL",
    start_angle=angle_start,
    custom_data=["NAME"],
    range_theta=[angle_start, angle_end],
    direction="counterclockwise",
)
fig.update_layout(
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    margin=dict(b=20, r=5, l=5, t=10),
    height=height,
    polar=dict(
        radialaxis=dict(
            showticklabels=False, ticks="", linecolor="rgba(255, 255, 255, 0)"
        ),
        angularaxis=dict(showticklabels=False, ticks="", linecolor="rgba(0,0,0,0)"),
    ),
    polar_radialaxis_gridcolor="rgba(0,0,0,0)",
    polar_angularaxis_gridcolor="rgba(0,0,0,0)",
)
fig.update_traces(textposition="middle center")
fig.update_traces(
    hovertemplate="<b>%{customdata[0]}</b><extra></extra>",
    textfont_size=12,
)
fig.update_layout(
    template=graph_style,
    autosize=True,
    showlegend=False,
    uniformtext_minsize=8,
    uniformtext_mode="hide",
    font=dict(family="Arial, monospace", size=12),
    width=600,
)
fig.update_traces(marker=dict(opacity=0.7, size=size_marker))
fig.add_annotation(x=0.5, y=0.2,text= str(len(df)), showarrow=False,font=dict(family="Arial, monospace",size=40))#annotation number
fig.show()